# Linear regression

This notebook compares various linear regression implementations. The dataset used is the [Toulouse bike sharing dataset](https://creme-ml.github.io/generated/creme.datasets.fetch_bikes.html#creme.datasets.fetch_bikes).

In [3]:
%load_ext watermark
%watermark --python --machine --packages creme,keras,sklearn,tensorflow,torch --datename

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Tue Oct 08 2019 

CPython 3.7.4
IPython 7.4.0

creme 0.3.0
keras 2.2.4
sklearn 0.21.3
tensorflow 1.14.0
torch 1.1.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.2.10-arch1-1-ARCH
machine    : x86_64
processor  : 
CPU cores  : 8
interpreter: 64bit


In [9]:
from creme import compose
from creme import datasets
from creme import feature_extraction
from creme import linear_model
from creme import metrics
from creme import optim
from creme import preprocessing
from creme import stats
from keras import layers
from keras import models
from keras import optimizers
from sklearn import linear_model as sk_linear_model
import torch

%run utils.py
%run wrappers.py

In [14]:
n_features = 6
lr = 0.005

class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.linear(x)
    
torch_model = PyTorchNet(n_features=n_features)

# Keras
inputs = layers.Input(shape=(n_features,))
predictions = layers.Dense(1, kernel_initializer='zeros', bias_initializer='zeros')(inputs)
keras_model = models.Model(inputs=inputs, outputs=predictions)
keras_model.compile(optimizer=optimizers.SGD(lr=lr), loss='mean_squared_error')


def add_hour(x):
    x['hour'] = x['moment'].hour
    return x

results = benchmark(
    get_X_y=datasets.fetch_bikes,
    n=182470,
    get_pp=lambda: (
        compose.Select('clouds', 'humidity', 'pressure', 'temperature', 'wind') +
        (
            add_hour |
            feature_extraction.TargetAgg(by=['station', 'hour'], how=stats.Mean())
        ) |
        preprocessing.StandardScaler()
    ),
    models=[
         ('creme', 'LinearRegression', linear_model.LinearRegression(
            optimizer=optim.SGD(lr),
            l2=0.,
            intercept_lr=lr
        )),
        ('scikit-learn', 'SGDRegressor', ScikitLearnRegressor(
            model=sk_linear_model.SGDRegressor(
                learning_rate='constant',
                eta0=lr,
                penalty='none'
            ),
        )),
        ('PyTorch (CPU)', 'Linear', PyTorchRegressor(
            network=torch_model,
            loss_fn=torch.nn.MSELoss(),
            optimizer=torch.optim.SGD(torch_model.parameters(), lr=lr)
        )),
        ('Keras on Tensorflow (CPU)', 'Dense', KerasRegressor(
            model=keras_model
        )),
        #('Vowpal Wabbit', '', VowpalWabbitRegressor(
        #    loss_function='squared',
        #    sgd=True,
        #    adaptive=False,
        #    normalized=False,
        #    invariant=False,
        #    initial_weight=0
        #))
    ],
    get_metric=metrics.MSE
)

In [15]:
results

,Library,Model,MSE,Fit time,Average fit time,Predict time,Average predict time
0,Vowpal Wabbit,,79.588114,"12s, 935ms, 475μs, 72ns","70μs, 891ns","26s, 721ms, 149μs, 367ns","146μs, 441ns"


In [7]:
results

,Library,Model,MSE,Fit time,Average fit time,Predict time,Average predict time
0,creme,LinearRegression,23.035085,"3s, 338ms, 465μs, 619ns","18μs, 296ns","892ms, 571μs, 786ns","4μs, 892ns"
1,scikit-learn,SGDRegressor,25.295369,"55s, 687ms, 660μs, 259ns","305μs, 188ns","19s, 876ms, 42μs, 73ns","108μs, 928ns"
2,PyTorch (CPU),Linear,23.035086,"1m, 45s, 290ms, 48μs, 287ns","577μs, 27ns","34s, 224ms, 26μs, 443ns","187μs, 560ns"
3,Keras on Tensorflow (CPU),Dense,23.035086,"3m, 40s, 447ms, 280μs, 203ns","1ms, 208μs, 129ns","2m, 11s, 758ms, 82μs, 511ns","722μs, 81ns"
